In [ ]:
import pandas as pd

sent_df = pd.read_csv('extracted_sentences_classified_1971_2022.csv')
sent_df = sent_df[['celex', 'sent', 'regulatory_according_to_rule', 'sent_count', 'word_count']]
metadata_df = pd.read_csv('metadata.csv')

In [ ]:
regulatory_df = sent_df[sent_df['regulatory_according_to_rule'] == 1.0]
constitutive_df = sent_df[sent_df['regulatory_according_to_rule'] == 0.0]
regulation_counts_df = regulatory_df.groupby(['celex'])['sent'].count()
rc_ind = regulation_counts_df.reset_index(drop=False)
rc = pd.DataFrame(rc_ind.values.tolist(), columns=['celex', 'reg_count'])

In [ ]:
col_reg_counts = []
sent_counts = []
word_counts = []
celexes = list(set(rc['celex'].tolist()))
count = 0
m_celexes = metadata_df['celex'].tolist()

idx = 0
for m_celex in m_celexes:
    idx += 1
    if idx % 1000 == 0:
        print(idx)

    if m_celex in celexes:
        matching_item = rc[rc['celex'] == m_celex]
        col_reg_counts.append(matching_item['reg_count'].tolist()[0])
        mat = sent_df[sent_df['celex'] == m_celex]
        sent_counts.append(mat['sent_count'].tolist()[0])
        word_counts.append(mat['word_count'].tolist()[0])
    else:
        col_reg_counts.append(0)
        mat = sent_df[sent_df['celex'] == m_celex]
        if len(mat) > 0:
            sent_counts.append(mat['sent_count'].tolist()[0])
            word_counts.append(mat['word_count'].tolist()[0])
        else:
            count += 1
            sent_counts.append(1)
            word_counts.append(1)

print(count)

In [ ]:
print(len(col_reg_counts))
print(len(sent_counts))
print(len(word_counts))

In [ ]:
metadata_df['reg_count'] = col_reg_counts
metadata_df['sent_count'] = sent_counts
metadata_df['word_count'] = word_counts
metadata_df.to_csv('metadata_enriched.csv', index=False)

In [26]:
import pandas as pd
df = pd.read_csv('metadata_enriched.csv')

Add a single date value (for adoption date) as a new column

In [28]:
canon_adoption_dates = []
print(len(df))
for index, row in df.iterrows():
    if len(row['date_adoption'].split('|')) > 1:
        cdate = row['date_adoption'].split('|')[0]
        curr_date = pd.to_datetime(cdate.strip(), format='%Y-%m-%d')
        canon_adoption_dates.append(curr_date)
    else:
        curr_date = pd.to_datetime(row['date_adoption'], format='%Y-%m-%d')
        canon_adoption_dates.append(curr_date)
        
print(len(canon_adoption_dates))

119077
119077


In [29]:
df['date'] = canon_adoption_dates
df.to_csv('metadata_enriched2.csv', index=False)

In [30]:
df

,celex,author,responsible_body,form,title,addressee,date_adoption,date_in_force,date_end_validity,directory_code,...,subject_matters,day,month,year,dc_string,fulltext_format,reg_count,sent_count,word_count,date
0,31971D0001,European Commission,NaN,D,NaN,Germany,1970-12-04,NaN,1971-12-31,NaN,...,Pigmeat | Information and verification | Europ...,4.0,12.0,1970.0,Agriculture,NaN,0,1,1,1970-12-04
1,31971D0002,European Commission,NaN,D,NaN,Belgium,1970-12-04,NaN,1971-12-31,NaN,...,Pigmeat | Information and verification | Europ...,4.0,12.0,1970.0,Agriculture,NaN,0,1,1,1970-12-04
2,31971D0003,European Commission,NaN,D,Decision No 3/71/ECSC of the Commission of 22 ...,France,1970-12-04,NaN,1971-12-31,NaN,...,Pigmeat | Information and verification | Europ...,4.0,12.0,1970.0,Agriculture,pdf,0,1,1,1970-12-04
3,31971D0004,European Commission,NaN,D,NaN,Italy,1970-12-04,NaN,1971-12-31,NaN,...,Pigmeat | Information and verification | Europ...,4.0,12.0,1970.0,Agriculture,NaN,0,1,1,1970-12-04
4,31971D0005,European Commission,NaN,D,NaN,Luxembourg,1970-12-04,NaN,1971-12-31,NaN,...,Pigmeat | Information and verification | Europ...,4.0,12.0,1970.0,Agriculture,NaN,0,1,1,1970-12-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119072,32022R2583,Council of the European Union,NaN,R,Council Regulation (EU) 2022/2583 of 19 Decemb...,NaN,2022-12-19,2022-12-31 | 2023-01-01,9999-12-31,http://publications.europa.eu/resource/authori...,...,Customs duties: suspensions | Common customs t...,19.0,12.0,2022.0,Customs Union and free movement of goods,html,0,1,1,2022-12-19
119073,32022R2584,Council of the European Union,NaN,R,Council Regulation (EU) 2022/2584 of 8 Novembe...,NaN,2022-11-08,2022-12-31 | 2022-11-08,9999-12-31,http://publications.europa.eu/resource/authori...,...,External relations | Fisheries policy,8.0,11.0,2022.0,Fisheries,html,1,1,32,2022-11-08
119074,32022R2586,Council of the European Union,NaN,R,Council Regulation (EU) 2022/2586 of 19 Decemb...,NaN,2022-12-19,2023-01-19,9999-12-31,http://publications.europa.eu/resource/authori...,...,Transport | State aids,19.0,12.0,2022.0,Competition policy,html,15,32,476,2022-12-19
119075,32022R2587,Directorate-General for Maritime Affairs and F...,NaN,R,COMMISSION DELEGATED REGULATION (EU) …/... ame...,NaN,2022-08-18,2022-12-31 | 2023-01-01,2025-12-31,http://publications.europa.eu/resource/authori...,...,Fisheries policy,18.0,8.0,2022.0,Fisheries,html,2,4,46,2022-08-18


In [ ]:
# calculate different metrics...TODO

# metric 1: raw count
# metric 2: raw count / word_count
metric_2 = []
# metric 3: raw_count / sent_count
metric_3 = []
# metric 4: (raw_count / sent_count) + (raw_count / mean_reg_count_year)
metric_4 = []
# metric 5: (raw_count / sent_count) + (raw_count / mean_reg_count)
metric_5 = []